## 필요한 라이브러리 Import

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

## 네이버증권 Financial Summary 크롤링

기본적으로 네이버증권은 URL에 종목코드륿 변수로 받아 종목코드별(기업별) 페이지를 구성하고 있다.<br><br>
Chrome 개발자 도구에 들어가 Network를 켜보면 "cF1001.aspx?cmp=cd"부분을 보면 매출액, 영업이익 등이 html 언어로 표현되어 있는 것을 알 수 있다. <br><br>
URL을 살펴보면 cmp_cd는 종목코드, freq_typ은 연간(A), 분기(Q), encparam과 id는 종목별로 랜덤하게 부여되는 key값 정도로 파악된다. <br><br>
encparam과 id는 랜덤 값으로 파악되기 때문에 이 정보를 알고 접근할 수는 없다. 그래서 2개의 값을 html에서 찾아서 접근하는 방법을 사용하기로 했다.

## encparam과 id 값 찾기

In [2]:
def get_basic_key_values(code):
    # 종목별 encparam, encid 찾기
    url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={code}'
    html = requests.get(url).text
    
    re_enc = re.compile("encparam: '(.*)'", re.IGNORECASE)
    re_id = re.compile("id: '([a-zA-Z0-9]*)' ?", re.IGNORECASE)
    
    encparam = re_enc.search(html).group(1)
    encid = re_id.search(html).group(1)
    
    return encparam, encid

get_basic_key_values('052860') # 종목코드 입력

('bjYrNEhwdWpnUnVvTDdnN3dycWk2UT09', 'QmZIZ20rMn')

## get_basic_key_values 함수를 사용해서 Financial Summary 값을 데이터 프레임으로 불러오기

In [3]:
def get_dataframe(code):
    
    encparam, encid = get_basic_key_values(code)

    url = f'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd={code}&fin_type=0&freq_type=A&encparam={encparam}&id={encid}'
    headers = {'Referer': 'HACK'}
    html = requests.get(url, headers=headers).text
    dfs = pd.read_html(html)
    df = dfs[1]

    #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ 데이터프레임 만지는 과정
    df = df.transpose() # 보기 편하게 데이터프레임의 행과 열을 바꾼다.

    find_url = f'https://finance.naver.com/item/main.naver?code={code}'

    response = requests.get(find_url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기업명 가져오기
    name_tag = soup.find('div', {'class': 'wrap_company'}).find('h2')
    stock_name = name_tag.text.strip()


    df['기업명'] = [stock_name, f'{stock_name}(2020/12)', f'{stock_name}(2021/12)', f'{stock_name}(2022/12)', f'{stock_name}(2023/12)', f'{stock_name}(2024/12)',
                  f'{stock_name}(2025/12)(예측)', f'{stock_name}(2026/12)(예측)', f'{stock_name}(2027/12)(예측)'] # 기업명 열을 추가한다.

    # '기업명' 열을 첫 번째 열로 이동시킨다.
    # 열 순서를 직접 지정한다.
    cols = ['기업명'] + [col for col in df.columns if col != '기업명']
    df = df[cols]

    # 첫 번째 행을 열 이름으로 설정하고 나머지만 남기기
    df.columns = df.iloc[0] # 첫 행을 column으로 지정한다.
    df = df[1:].reset_index(drop=True) # 첫 행을 삭제하고 인덱스를 초기화한다.
    df = df.rename(columns={df.columns[0]: '기업명'}) # 첫 번째 열 이름을 '기업명'으로 변경


    return df

get_dataframe('052860') # 종목 코드 입력

/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


"(주요재무정보, 주요재무정보)",기업명,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
0,아이앤씨(2020/12),229.0,-56.0,-56.0,-60.0,-60.0,-61.0,1.0,630.0,153.0,...,32.01,472.8,-347.0,NaN,2767.0,1.55,0.0,0.0,0.0,17694054.0
1,아이앤씨(2021/12),299.0,-35.0,-35.0,-54.0,-54.0,-55.0,0.0,649.0,220.0,...,51.08,417.01,-307.0,NaN,2465.0,1.61,NaN,NaN,0.0,17862854.0
2,아이앤씨(2022/12),394.0,24.0,24.0,10.0,10.0,10.0,1.0,654.0,211.0,...,47.55,430.95,54.0,46.27,2537.0,0.98,NaN,NaN,0.0,17862854.0
3,아이앤씨(2023/12),643.0,14.0,14.0,1.0,0.0,-3.0,3.0,673.0,227.0,...,51.11,427.91,-17.0,NaN,2529.0,1.09,NaN,NaN,0.0,17862854.0
4,아이앤씨(2024/12),234.0,-107.0,-107.0,-120.0,-120.0,-119.0,-1.0,513.0,208.0,...,68.27,297.66,-668.0,NaN,1836.0,0.98,NaN,NaN,NaN,17862854.0
5,아이앤씨(2025/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,아이앤씨(2026/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,아이앤씨(2027/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 특정 섹터에 있는 종목들의 종목 코드 크롤링

In [6]:
# 예시 URL (업종 리스트 페이지로 대체 가능)
url = 'https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no=278'  # 반도체와반도체장비 업종

# 요청 보내기 (User-Agent 포함)
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 종목 코드 추출
stock_codes = []

for tag in soup.select('td.name a'):
    href = tag.get('href')
    if 'code=' in href:
        code = href.split('code=')[-1]
        stock_codes.append(code)

# 결과 출력
print(stock_codes)

['136660', '356860', '219130', '031980', '464500', '212710', '036930', '092070', '241790', '417500', '045300', '089970', '319660', '429270', '311320', '451220', '064520', '054620', '096610', '149010', '172670', '222160', '272110', '122640', '101400', '011930', '087600', '054940', '030530', '183300', '227950', '241770', '102710', '083310', '425420', '003160', '005935', '079370', '445180', '059090', '082270', '005930', '029460', '226590', '140070', '112290', '061970', '084370', '067310', '264660', '330860', '142210', '101160', '217500', '086390', '094170', '114810', '131970', '102120', '080220', '052900', '382800', '195870', '160980', '394280', '389020', '064760', '058470', '232140', '033160', '425040', '083450', '093520', '053610', '166090', '101490', '240810', '420770', '405100', '033640', '348350', '396270', '005290', '071280', '098120', '169670', '080520', '365590', '445090', '032580', '000660', '036540', '036200', '171010', '117670', '054450', '089890', '042700', '147760', '322310',

### 코스피 200에 편입되어 있는 200개의 종목들의 종목 코드 크롤링

In [6]:
BaseUrl = 'http://finance.naver.com/sise/entryJongmok.nhn?&page='

# 종목 코드 추출
stock_codes = []

for i in range(1, 21):  # range() 함수의 끝 파라미터는 포함되지 않는다.
    url = BaseUrl + str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    items = soup.find_all('td', {'class': 'ctg'})
    for item in items:
        txt = item.a.get('href')  # 예: https://finance.naver.com/item/main.nhn?code=006390
        k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복
        if k:
            code = k.group()
            stock_codes.append(code)

print(stock_codes)

<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/155905241.py:13: SyntaxWarning: invalid escape sequence '\d'
  k = re.search('[\d]+', txt)  # 정규표현식 사용. [\d] 숫자표현, + : 반복


['005930', '000660', '373220', '207940', '005380', '012450', '068270', '000270', '329180', '105560', '035420', '055550', '012330', '042660', '138040', '005490', '028260', '009540', '034020', '086790', '259960', '000810', '032830', '035720', '010130', '015760', '011200', '051910', '096770', '033780', '316140', '030200', '010140', '064350', '024110', '006400', '402340', '066570', '017670', '352820', '267260', '323410', '003550', '018260', '003670', '034730', '000100', '009150', '047050', '086280', '326030', '047810', '003490', '272210', '042700', '003230', '090430', '079550', '006800', '010620', '005830', '021240', '267250', '010120', '010950', '180640', '051900', '032640', '009830', '161390', '005940', '000150', '271560', '029780', '241560', '016360', '000720', '071050', '298040', '377300', '034220', '006260', '450080', '011790', '251270', '028050', '022100', '001040', '000880', '097950', '078930', '036460', '035250', '128940', '039490', '175330', '011070', '138930', '004020', '454910',

### 데이터프레임 합치기 (크롤링한 데이터를 하나의 데이터프레임으로 결합)

In [7]:
# 종목 코드 리스트를 사용하여 데이터프레임 생성
final_df = pd.DataFrame() # 빈 데이터프레임 생성

for code in stock_codes:
    try:
        df = get_dataframe(code) # 각 종목 코드에 대해 데이터프레임 생성
        final_df = pd.concat([final_df, df], axis=0) # 데이터프레임 합치기
    except Exception as e:
        print(f"Error processing code {code}: {e}")

final_df.reset_index(drop=True, inplace=True) # 인덱스 초기화
final_df # 최종 데이터프레임 출력

/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
/var/folders/vw/vspwd_yn6b94yd37dg899j380000gn/T/ipykernel_43462/3426255862.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed

"(주요재무정보, 주요재무정보)",기업명,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
0,삼성전자(2020/12),2368070.0,359939.0,359939.0,363451.0,264078.0,260908.0,3170.0,3782357.0,1022877.0,...,37.07,30692.79,3841.0,21.09,39406.0,2.06,2994.0,3.7,77.95,5969782550.0
1,삼성전자(2021/12),2796048.0,516339.0,516339.0,533518.0,399074.0,392438.0,6637.0,4266212.0,1217212.0,...,39.92,33143.62,5777.0,13.55,43611.0,1.8,1444.0,1.84,25.0,5969782550.0
2,삼성전자(2022/12),3022314.0,433766.0,433766.0,464405.0,556541.0,547300.0,9241.0,4484245.0,936749.0,...,26.41,38144.29,8057.0,6.86,50817.0,1.09,1444.0,2.61,17.92,5969782550.0
3,삼성전자(2023/12),2589355.0,65670.0,65670.0,110063.0,154871.0,144734.0,10137.0,4559060.0,922281.0,...,25.36,39114.28,2131.0,36.84,52002.0,1.51,1444.0,1.84,67.78,5969782550.0
4,삼성전자(2024/12),3008709.0,327260.0,327260.0,375297.0,344514.0,336214.0,8300.0,5145319.0,1123399.0,...,27.93,41772.84,4950.0,10.75,57981.0,0.92,1446.0,2.72,29.18,5969782550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,삼아알미늄(2023/12),2680.0,38.0,38.0,12.0,34.0,34.0,NaN,4043.0,1493.0,...,58.56,3366.42,236.0,451.3,17332.0,6.14,100.0,0.09,43.19,14711916.0
1596,삼아알미늄(2024/12),2517.0,-96.0,-96.0,-103.0,-94.0,-94.0,NaN,4362.0,1926.0,...,79.04,3211.74,-636.0,NaN,16559.0,2.04,25.0,0.07,-3.93,14711916.0
1597,삼아알미늄(2025/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,삼아알미늄(2026/12)(예측),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 최종 데이터프레임을 csv파일로 변환

In [8]:
final_df.to_csv('financial_summary_cospi200.csv', index=False, encoding='utf-8-sig')